In [1]:
# test edit
# based on https://mybinder.org/v2/gh/projectnessie/nessie-demos/main?filepath=notebooks/nessie-iceberg-demo-nba.ipynb

import findspark
import os

findspark.init(spark_home='/Users/rhyando/spark')
os.environ['JAVA_HOME']='/opt/homebrew/Cellar/openjdk@11/11.0.23/libexec/openjdk.jdk/Contents/Home'
# os.environ['SPARK_LOCAL_IP']='192.168.49.2'
import pyspark
from pyspark.sql import SparkSession
import os
import socket

# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.82.0'

conf = pyspark.SparkConf().setAll([
    # ip of spark master
    # ('spark.driver.host', '0.0.0.0')
    # , ('spark.driver.bindAddress', '0.0.0.0')
    # ('spark.driver.host', socket.gethostbyname(socket.gethostname()))
    #, ('spark.ui.proxyBase', os.environ['JUPYTERHUB_SERVICE_PREFIX'] + 'proxy/4040')
    # ('spark.submit.deployMode', 'client')
    # , ('spark.driver.port', '7887')
    ('spark.master', "spark://127.0.0.1:7077")
    # , ('spark.jars.packages', 'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.82.0,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-aws:3.3.4')
    , ("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions")
    # cluster ip
    # , ('spark.hadoop.fs.s3a.endpoint', 'http://10.104.71.148:6544')
    # , ('spark.hadoop.fs.s3a.endpoint', "http://minio-service.minio-dev.svc.cluster.local:6544")
    # for minio spark
    , ('spark.hadoop.fs.s3a.access.key','minio')
    , ('spark.hadoop.fs.s3a.secret.key', 'minio123')
    , ('spark.hadoop.fs.s3a.path.style.access', 'true')
    , ("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    , ("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    , ('spark.hadoop.fs.s3a.endpoint', "http://127.0.0.1:6544")
    # for iceberg minio jdbc
    #, ('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebesrgSparkSessionExtensions')
    , ('spark.sql.catalog.nessie_catalog', 'org.apache.iceberg.spark.SparkCatalog')
    , ("spark.sql.catalog.nessie_catalog.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    , ('spark.sql.defaultCatalog', 'nessie_catalog')
    , ('spark.sql.catalog.nessie_catalog.warehouse', 's3a://iceberg/')
    # , ('spark.sql.catalog.my_catalog.type', 'jdbc')
    # ip of service postgre
    , ('spark.sql.catalog.nessie_catalog.uri', 'http://127.0.0.1:6788/api/v1')
    # , ('spark.sql.catalog.nessie_catalog.uri', 'http://nessie-service.nessie-dev.svc.cluster.local:6788/api/v1')
    , ('spark.sql.catalog.nessie_catalog.ref', 'main')
    , ("spark.sql.catalog.nessie_catalog.authentication.type", 'NONE')
    # , ('spark.sql.catalog.my_catalog.jdbc.user', 'postgres')
    # , ('spark.sql.catalog.my_catalog.jdbc.password', 'postgres')
    ])
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

24/06/04 20:30:49 WARN Utils: Your hostname, Rhyandos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.178.241 instead (on interface en0)
24/06/04 20:30:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/04 20:30:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import os
os.getenv('SPARK_LOCAL_IP')

In [3]:
socket.gethostbyname(socket.gethostname())

'127.0.0.1'

In [4]:
spark.sql("LIST REFERENCES IN nessie_catalog").show()

+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch| dev|377f3500db1793fcf...|
| Branch|main|677a4462c3b296f12...|
+-------+----+--------------------+



In [4]:
spark.sql(
    """CREATE TABLE IF NOT EXISTS nessie_catalog.datasaku.salaries
            (Season STRING, Team STRING, Salary STRING, Player STRING)"""
).show()


24/06/03 21:51:52 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


++
||
++
++



In [3]:
spark.sql(
    """CREATE TABLE IF NOT EXISTS nessie_catalog.datasaku.salaries_test
            (Season STRING, Team STRING, Salary STRING, Player STRING)"""
).show()


24/06/04 19:27:40 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


++
||
++
++



In [6]:
spark.sql("USE REFERENCE dev IN nessie_catalog").show()
spark.sql("SHOW REFERENCE IN nessie_catalog").show()
spark.sql("SHOW TABLES IN nessie_catalog").show()

+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch| dev|377f3500db1793fcf...|
+-------+----+--------------------+

+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch| dev|377f3500db1793fcf...|
+-------+----+--------------------+

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
| datasaku|     salaries|      false|
| datasaku|salaries_test|      false|
+---------+-------------+-----------+



In [3]:
spark.sql("INSERT INTO nessie_catalog.datasaku.salaries VALUES ('1', 'bulls', '50', 'kurdo'), ('2', 'bulls', '51', 'kurdo');")

In [4]:
spark.sql(
    """select * from nessie_catalog.datasaku.salaries"""
).show()


24/06/04 20:30:55 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+------+----+------+------+
|Season|Team|Salary|Player|
+------+----+------+------+
+------+----+------+------+



# try package

In [ ]:
from datasaku import datasaku_spark

ModuleNotFoundError: No module named 'datasaku'